# Class 정의

In [14]:
!python.exe -m pip install --upgrade pip
!pip install python-dotenv
!pip install google-generativeai
!pip install mysql-connector-python

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
  Using cached mysql_connector_python-9.2.0-cp313-cp313-win_amd64.whl.metadata (6.2 kB)
Using cached mysql_connector_python-9.2.0-cp313-cp313-win_amd64.whl (16.1 MB)


In [ ]:
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai
import mysql.connector
from mysql.connector import errors

class NovelGenerator:
    def __init__(self, genre: str, title: str):
        # 환경변수 로드 및 API, DB 설정
        load_dotenv()
        self.GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        genai.configure(api_key=self.GEMINI_API_KEY)

        self.genre = genre
        self.title = title
        self.worldview = ""
        self.synopsis = ""
        self.characters = ""
        self.first_chapter = ""
        self.next_chapter = ""
        
        # 여기서는 간단한 예시로 단일 소설에 대해 novel_pk를 1로 고정
        self.novel_pk = 1
        
        # DB 접속정보
        self.db_host = os.getenv("DB_HOST")
        self.db_port = int(os.getenv("DB_PORT", "3306"))
        self.db_user = os.getenv("DB_USER")
        self.db_password = os.getenv("DB_PASSWORD")
        self.db_name = os.getenv("DB_NAME")
        
        # 데이터베이스가 없는 경우 생성하도록 처리
        try:
            self.db_connection = mysql.connector.connect(
                host=self.db_host,
                port=self.db_port,
                user=self.db_user,
                password=self.db_password,
                database=self.db_name
            )
        except errors.ProgrammingError as e:
            if e.errno == 1049:  # Unknown database
                print(f"데이터베이스 {self.db_name}가 존재하지 않습니다. 데이터베이스를 생성합니다.")
                temp_conn = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password
                )
                temp_cursor = temp_conn.cursor()
                temp_cursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.db_name}")
                temp_conn.commit()
                temp_cursor.close()
                temp_conn.close()
                # 데이터베이스 생성 후 다시 연결
                self.db_connection = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password,
                    database=self.db_name
                )
            else:
                raise e
        
        self.create_table_if_not_exists()

    def create_table_if_not_exists(self):
        """episode 테이블이 없으면 생성"""
        cursor = self.db_connection.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS episode (
            ep_pk INT AUTO_INCREMENT PRIMARY KEY,
            ep_title VARCHAR(255) NOT NULL,
            novel_pk INT NOT NULL,
            created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
            ep_content TEXT NOT NULL
        )
        """
        cursor.execute(query)
        self.db_connection.commit()
        cursor.close()

    def insert_episode(self, ep_title: str, ep_content: str):
        """새 에피소드를 DB에 저장합니다."""
        cursor = self.db_connection.cursor()
        query = """
        INSERT INTO episode (ep_title, novel_pk, ep_content)
        VALUES (%s, %s, %s)
        """
        cursor.execute(query, (ep_title, self.novel_pk, ep_content))
        self.db_connection.commit()
        cursor.close()

    def get_previous_episodes(self) -> str:
        """해당 소설의 모든 에피소드(ep_content)를 created_date 기준 오름차순으로 불러와 하나의 문자열로 합칩니다."""
        cursor = self.db_connection.cursor()
        query = """
        SELECT ep_content FROM episode
        WHERE novel_pk = %s
        ORDER BY created_date ASC
        """
        cursor.execute(query, (self.novel_pk,))
        rows = cursor.fetchall()
        cursor.close()
        episodes = "\n\n---\n\n".join([row[0] for row in rows])
        return episodes

    def recommend_worldview(self) -> str:
        """소설 세계관 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 세계관을 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목을 기반으로 독창적이고 생동감 있는 소설 세계관을 만들어주세요.
        소설 세계관을 구성할 때, 아래의 항목들을 참고하여 상세하게 작성하세요.

        1. 기본 설정
            - 시대: (예: 과거, 현대, 미래 등) - 시대적 배경과 분위기를 구체적으로 설명해주세요.
            - 핵심 테마: (예: 인간과 자연의 대립, 기술과 마법의 공존 등) - 세계관을 관통하는 핵심 주제를 명확히 제시하고, 다른 요소들과의 연관성을 설명해주세요.

        2. 지리적 배경
            - 세계 전체의 구조(대륙, 국가, 도시, 마을 등) - 지리적 구조를 시각적으로 표현하고, 각 지역의 특징을 설명해주세요.
            - 주요 지형 및 자연 환경(기후, 산맥, 강 등) - 자연 환경이 세계관에 미치는 영향을 설명하고, 상징적인 의미를 부여할 수 있습니다.
            - 상징적이거나 중요한 장소(예: 전설적인 성, 신비로운 숲 등) - 각 장소의 역사, 특징, 중요성 등을 구체적으로 설명해주세요.

        3. 역사와 문화
            - 세계의 건국 신화 및 주요 역사적 사건 - 역사적 사건이 세계관에 미친 영향과 현재 사회에 남은 유산을 설명해주세요.
            - 사회 구조(정치 체제, 경제 시스템, 계층 구조 등) - 사회 구조가 세계관 구성원들의 삶에 미치는 영향을 설명해주세요.
            - 문화와 종교(전통, 의식, 종교적 신념 등) - 문화와 종교가 세계관에 미치는 영향을 설명하고, 독특한 요소들을 강조해주세요.

        4. 기술 및 초자연적 요소
            - 세계 내 기술 수준(과거, 현대, 미래 기술) - 기술 수준이 세계관에 미치는 영향과 특징을 설명해주세요.
            - 만약 존재한다면 마법이나 초자연적 현상의 규칙과 한계 - 마법/초자연 현상이 세계관에 미치는 영향과 작동 방식을 구체적으로 설명해주세요.

        5. 인물 및 종족
            - 주요 인물이나 집단, 종족의 특징과 역할 - 각 인물/종족의 개성과 세계관 내 역할을 명확하게 설명해주세요.
            - 각 인물/종족이 속한 사회적, 문화적 배경 - 인물/종족의 배경이 그들의 행동 양식에 미치는 영향을 설명해주세요.

        6. 갈등 구조
            - 사회 내부 혹은 외부의 갈등 요소(정치적 분쟁, 종족 간 충돌 등) - 갈등의 원인, 진행 과정, 결과를 구체적으로 설명해주세요.
            - 이러한 갈등이 전체 세계관에 미치는 영향 - 갈등이 세계관의 다른 요소들에 미치는 영향을 설명해주세요.

        7. 세부 설정 및 고유 용어
            - 세계관 내에서만 사용되는 독특한 용어나 법칙 정리 - 용어의 의미와 사용 예시를 설명하여 세계관의 현실성을 높여주세요.
            - 설정의 일관성을 유지할 수 있도록 참고 자료 작성 - 필요한 경우, 추가적인 자료를 작성하여 세계관의 완성도를 높여주세요.
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n\n**소설 세계관**\n"
        response = model.generate_content(prompt)
        self.worldview = response.text
        print("소설 세계관:\n", self.worldview)
        return self.worldview

    def recommend_synopsis(self) -> str:
        """소설 줄거리 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 줄거리를 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목, 세계관을 기반으로 독창적이고 생동감 있는 소설 줄거리를 만들어주세요.
        소설 줄거리를 구성할 때, 아래의 항목들을 참고하세요.

        1. 기본 정보
            - 장르: [예: 판타지, SF 등]
            - 제목: [제목 입력]
            - 세계관: [세계관의 배경, 시대, 문화, 기술 등 간단히 설명]

        2. 도입부
            - 소설이 펼쳐지는 세계관 소개
            - 주요 인물 및 그들의 기본 목표
            - 초기 갈등이나 문제의 암시

        3. 전개부
            - 주된 갈등 및 도전 과제 전개
            - 인물 간의 관계와 서브 플롯 설명
            - 주인공이 맞서야 하는 문제와 성장 과정

        4. 클라이맥스
            - 갈등의 최고조와 결정적 순간
            - 주인공의 중요한 선택 및 대립의 절정

        5. 결말
            - 갈등 해결 및 인물 변화
            - 세계관에 미친 영향과 여운 남기기

        위 템플릿에 따라 소설의 줄거리를 상세하게 작성해 주세요. 
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n## 소설 세계관: {self.worldview}\n\n**소설 줄거리**\n"
        response = model.generate_content(prompt)
        self.synopsis = response.text
        print("소설 줄거리:\n", self.synopsis)
        return self.synopsis

    def recommend_characters(self) -> str:
        """소설 등장인물 생성 함수"""
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리를 기반으로 소설 등장인물을 만들어주세요.

        각 등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
            - (예: 키 180cm, 날카로운 눈매, 과묵한 성격 등) - 외모, 성격, 능력 등 세부 묘사 (선택)
            - (예: 특정 능력, 습관, 버릇, 가치관 등) - 등장인물의 개성을 드러내는 특징 (선택)
            - (예: 가문, 출신, 과거 등) - 등장인물의 과거와 배경 (선택)
            - (예: 주인공과 친구, 연인 관계 등) - 다른 등장인물과의 관계 (선택)

        여러 명의 등장인물을 생성할 수 있으며, 각 등장인물은 아래와 같은 형태로 표현되어야 합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            },
            {
                "이름": "춘향이",
                "성별": "여",
                "나이": "18",
                "역할": "조력자",
                "직업": "농부",
                "프로필": "밝고 활발한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**새로운 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        new_characters = response.text
        new_characters = re.sub(r'```json\n(.*?)\n```', r'\1', new_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(new_characters)
        except Exception:
            new_char = new_characters
        # 수정: new_char가 리스트인 경우 extend, 그렇지 않으면 append
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물:\n", self.characters)
        return self.characters

    def add_new_characters(self) -> str:
        """
        기존 등장인물에 추가로 새로운 등장인물을 생성하는 함수.
        기존 캐릭터와 차별화된 새로운 인물들을 생성하여 기존 목록 리스트에 덧붙입니다.
        """
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리, 기존 등장인물들을 기반으로 기존과 다른 새로운 소설 등장인물을 만들어주세요.

        등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
        등장인물은 아래와 같은 형태로 표현되어야 하며, 한 명의 캐릭터를 생성합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**추가 생성된 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        additional_characters = response.text
        additional_characters = re.sub(r'```json\n(.*?)\n```', r'\1', additional_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(additional_characters)
        except Exception:
            new_char = additional_characters
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물 업데이트:\n", self.characters)
        return self.characters

    def create_episode(self) -> str:
        """
        DB에 저장된 이전 에피소드들을 모두 불러와서, 
        이를 기반으로 새 에피소드(초안 또는 다음 화)를 생성합니다.
        각 에피소드는 500-1000자 정도로 작성합니다.
        """
        previous_episodes = self.get_previous_episodes()
        
        if not previous_episodes:
            # 이전 에피소드가 없다면 첫 번째 에피소드(초안) 생성
            instruction = """
            당신은 창의적이고 독창적인 소설 작가입니다. 
            주어진 장르, 제목, 세계관, 줄거리, 등장인물을 기반으로 소설의 초안을 작성해야 합니다.
            각 에피소드는 500-1000자 정도로 작성해야 합니다.

            <작성 지침>
            - 소설의 분위기는 주어진 장르에 맞게 설정하세요.
            - 첫 번째 에피소드에서는 주인공을 등장시키고, 소설의 시작점을 설정하세요.
            - 분량을 준수하되, 에피소드가 끝날 때 문장이 완결되도록 작성하세요.
            """
            episode_label = "**소설 초안**\n"
        else:
            # 이전 에피소드가 있다면 DB의 모든 내용을 기반으로 다음 화 생성
            instruction = """
            당신은 창의적이고 독창적인 소설 작가입니다. 
            주어진 장르, 제목, 세계관, 줄거리, 등장인물, 소설 이전화를 기반으로 소설의 다음화를 작성해야 합니다.
            각 에피소드는 500-1000자 정도로 작성해야 합니다.

            <작성 지침>
            - 소설의 분위기는 주어진 장르에 맞게 설정하세요.
            - 이전화의 내용을 참고하여 내용이 자연스럽게 연결되도록 작성하세요.
            - 분량을 준수하되, 에피소드가 끝날 때 문장이 완결되도록 작성하세요.
            """
            episode_label = "**소설 다음화**\n"
        
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 소설 등장인물: {self.characters}\n"
        )
        if previous_episodes:
            prompt += f"## 소설 초안: {previous_episodes}\n\n"
        prompt += episode_label

        response = model.generate_content(prompt)
        episode_content = response.text

        # 에피소드 번호는 기존 에피소드 수 + 1
        cursor = self.db_connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM episode WHERE novel_pk = %s", (self.novel_pk,))
        (count,) = cursor.fetchone()
        cursor.close()
        ep_number = count + 1
        ep_title = f"Episode {ep_number}"

        # 새로 생성된 에피소드를 DB에 저장
        self.insert_episode(ep_title, episode_content)

        if ep_number == 1:
            self.first_episode = episode_content
            print("에피소드 1화:\n", episode_content)
        else:
            self.next_episode = episode_content
            print(f"에피소드 {ep_number}화:\n", episode_content)
        return episode_content


# 테스트용 Class

In [ ]:
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai
import mysql.connector
from mysql.connector import errors

class NovelGenerator:
    def __init__(self, genre: str, title: str):
        # 환경변수 로드 및 API, DB 설정
        load_dotenv()
        self.GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        genai.configure(api_key=self.GEMINI_API_KEY)

        self.genre = genre
        self.title = title
        self.worldview = ""
        self.synopsis = ""
        self.characters = ""
        self.first_chapter = ""
        self.next_chapter = ""
        
        # 여기서는 간단한 예시로 단일 소설에 대해 novel_pk를 1로 고정
        self.novel_pk = 1
        
        # DB 접속정보
        self.db_host = os.getenv("DB_HOST")
        self.db_port = int(os.getenv("DB_PORT", "3306"))
        self.db_user = os.getenv("DB_USER")
        self.db_password = os.getenv("DB_PASSWORD")
        self.db_name = os.getenv("DB_NAME")
        
        # 데이터베이스가 없는 경우 생성하도록 처리
        try:
            self.db_connection = mysql.connector.connect(
                host=self.db_host,
                port=self.db_port,
                user=self.db_user,
                password=self.db_password,
                database=self.db_name
            )
        except errors.ProgrammingError as e:
            if e.errno == 1049:  # Unknown database
                print(f"데이터베이스 {self.db_name}가 존재하지 않습니다. 데이터베이스를 생성합니다.")
                temp_conn = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password
                )
                temp_cursor = temp_conn.cursor()
                temp_cursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.db_name}")
                temp_conn.commit()
                temp_cursor.close()
                temp_conn.close()
                # 데이터베이스 생성 후 다시 연결
                self.db_connection = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password,
                    database=self.db_name
                )
            else:
                raise e
        
        self.create_table_if_not_exists()

    def create_table_if_not_exists(self):
        """episode 테이블이 없으면 생성"""
        cursor = self.db_connection.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS episode (
            ep_pk INT AUTO_INCREMENT PRIMARY KEY,
            ep_title VARCHAR(255) NOT NULL,
            novel_pk INT NOT NULL,
            created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
            ep_content TEXT NOT NULL
        )
        """
        cursor.execute(query)
        self.db_connection.commit()
        cursor.close()

    def insert_episode(self, ep_title: str, ep_content: str):
        """새 에피소드를 DB에 저장합니다."""
        cursor = self.db_connection.cursor()
        query = """
        INSERT INTO episode (ep_title, novel_pk, ep_content)
        VALUES (%s, %s, %s)
        """
        cursor.execute(query, (ep_title, self.novel_pk, ep_content))
        self.db_connection.commit()
        cursor.close()

    def get_previous_episodes(self) -> str:
        """해당 소설의 모든 에피소드(ep_content)를 created_date 기준 오름차순으로 불러와 하나의 문자열로 합칩니다."""
        cursor = self.db_connection.cursor()
        query = """
        SELECT ep_content FROM episode
        WHERE novel_pk = %s
        ORDER BY created_date ASC
        """
        cursor.execute(query, (self.novel_pk,))
        rows = cursor.fetchall()
        cursor.close()
        episodes = "\n\n---\n\n".join([row[0] for row in rows])
        return episodes

    def recommend_worldview(self) -> str:
        """소설 세계관 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 세계관을 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목을 기반으로 독창적이고 생동감 있는 소설 세계관을 만들어주세요.
        소설 세계관을 구성할 때, 아래의 항목들을 참고하여 상세하게 작성하세요.

        1. 기본 설정
            - 시대: (예: 과거, 현대, 미래 등) - 시대적 배경과 분위기를 구체적으로 설명해주세요.
            - 핵심 테마: (예: 인간과 자연의 대립, 기술과 마법의 공존 등) - 세계관을 관통하는 핵심 주제를 명확히 제시하고, 다른 요소들과의 연관성을 설명해주세요.

        2. 지리적 배경
            - 세계 전체의 구조(대륙, 국가, 도시, 마을 등) - 지리적 구조를 시각적으로 표현하고, 각 지역의 특징을 설명해주세요.
            - 주요 지형 및 자연 환경(기후, 산맥, 강 등) - 자연 환경이 세계관에 미치는 영향을 설명하고, 상징적인 의미를 부여할 수 있습니다.
            - 상징적이거나 중요한 장소(예: 전설적인 성, 신비로운 숲 등) - 각 장소의 역사, 특징, 중요성 등을 구체적으로 설명해주세요.

        3. 역사와 문화
            - 세계의 건국 신화 및 주요 역사적 사건 - 역사적 사건이 세계관에 미친 영향과 현재 사회에 남은 유산을 설명해주세요.
            - 사회 구조(정치 체제, 경제 시스템, 계층 구조 등) - 사회 구조가 세계관 구성원들의 삶에 미치는 영향을 설명해주세요.
            - 문화와 종교(전통, 의식, 종교적 신념 등) - 문화와 종교가 세계관에 미치는 영향을 설명하고, 독특한 요소들을 강조해주세요.

        4. 기술 및 초자연적 요소
            - 세계 내 기술 수준(과거, 현대, 미래 기술) - 기술 수준이 세계관에 미치는 영향과 특징을 설명해주세요.
            - 만약 존재한다면 마법이나 초자연적 현상의 규칙과 한계 - 마법/초자연 현상이 세계관에 미치는 영향과 작동 방식을 구체적으로 설명해주세요.

        5. 인물 및 종족
            - 주요 인물이나 집단, 종족의 특징과 역할 - 각 인물/종족의 개성과 세계관 내 역할을 명확하게 설명해주세요.
            - 각 인물/종족이 속한 사회적, 문화적 배경 - 인물/종족의 배경이 그들의 행동 양식에 미치는 영향을 설명해주세요.

        6. 갈등 구조
            - 사회 내부 혹은 외부의 갈등 요소(정치적 분쟁, 종족 간 충돌 등) - 갈등의 원인, 진행 과정, 결과를 구체적으로 설명해주세요.
            - 이러한 갈등이 전체 세계관에 미치는 영향 - 갈등이 세계관의 다른 요소들에 미치는 영향을 설명해주세요.

        7. 세부 설정 및 고유 용어
            - 세계관 내에서만 사용되는 독특한 용어나 법칙 정리 - 용어의 의미와 사용 예시를 설명하여 세계관의 현실성을 높여주세요.
            - 설정의 일관성을 유지할 수 있도록 참고 자료 작성 - 필요한 경우, 추가적인 자료를 작성하여 세계관의 완성도를 높여주세요.
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n\n**소설 세계관**\n"
        response = model.generate_content(prompt)
        self.worldview = response.text
        print("소설 세계관:\n", self.worldview)
        return self.worldview

    def recommend_synopsis(self) -> str:
        """소설 줄거리 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 줄거리를 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목, 세계관을 기반으로 독창적이고 생동감 있는 소설 줄거리를 만들어주세요.
        소설 줄거리를 구성할 때, 아래의 항목들을 참고하세요.

        1. 기본 정보
            - 장르: [예: 판타지, SF 등]
            - 제목: [제목 입력]
            - 세계관: [세계관의 배경, 시대, 문화, 기술 등 간단히 설명]
        
        2. 발단
            - 소설이 펼쳐지는 세계관의 기초 소개
            - 주요 인물들의 소개 및 그들이 추구하는 기본 목표
            - 이야기에 숨은 첫 갈등이나 문제의 암시
        
        3. 전개
            - 주된 갈등 및 도전 과제의 본격적인 전개
            - 인물들 간의 관계, 서브 플롯, 그리고 각 인물의 내면적 변화
            - 주인공이 맞서야 하는 문제와 성장 과정을 상세히 설명
        
        4. 위기
            - 갈등이 극한으로 치닫는 순간, 상황이 악화되고 긴장이 고조됨
            - 주인공과 주변 인물들이 맞닥뜨리는 결정적 어려움 및 시련
            - 이야기의 전환점 역할을 하는 중요한 사건 발생
        
        5. 절정
            - 갈등의 최고조, 결정적인 대립과 순간의 전환
            - 주인공의 중대한 선택 및 운명을 건 마지막 대결
            - 전체 이야기의 긴장감과 감정이 폭발하는 순간
        
        6. 결말
            - 갈등의 해소와 함께 인물들의 변화 및 성장 결과 제시
            - 사건이 세계관에 미친 영향과 이후의 여운 남기는 마무리
            - 독자에게 남기는 메시지 혹은 후일담

        위 템플릿에 따라 소설의 줄거리를 상세하게 작성해 주세요. 
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n## 소설 세계관: {self.worldview}\n\n**소설 줄거리**\n"
        response = model.generate_content(prompt)
        self.synopsis = response.text
        print("소설 줄거리:\n", self.synopsis)
        return self.synopsis

    def recommend_characters(self) -> str:
        """소설 등장인물 생성 함수"""
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리를 기반으로 소설 등장인물을 만들어주세요.

        각 등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
            - (예: 키 180cm, 날카로운 눈매, 과묵한 성격 등) - 외모, 성격, 능력 등 세부 묘사 (선택)
            - (예: 특정 능력, 습관, 버릇, 가치관 등) - 등장인물의 개성을 드러내는 특징 (선택)
            - (예: 가문, 출신, 과거 등) - 등장인물의 과거와 배경 (선택)
            - (예: 주인공과 친구, 연인 관계 등) - 다른 등장인물과의 관계 (선택)

        여러 명의 등장인물을 생성할 수 있으며, 각 등장인물은 아래와 같은 형태로 표현되어야 합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            },
            {
                "이름": "춘향이",
                "성별": "여",
                "나이": "18",
                "역할": "조력자",
                "직업": "농부",
                "프로필": "밝고 활발한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**새로운 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        new_characters = response.text
        new_characters = re.sub(r'```json\n(.*?)\n```', r'\1', new_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(new_characters)
        except Exception:
            new_char = new_characters
        # 수정: new_char가 리스트인 경우 extend, 그렇지 않으면 append
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물:\n", self.characters)
        return self.characters

    def add_new_characters(self) -> str:
        """
        기존 등장인물에 추가로 새로운 등장인물을 생성하는 함수.
        기존 캐릭터와 차별화된 새로운 인물들을 생성하여 기존 목록 리스트에 덧붙입니다.
        """
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리, 기존 등장인물들을 기반으로 기존과 다른 새로운 소설 등장인물을 만들어주세요.

        등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
        등장인물은 아래와 같은 형태로 표현되어야 하며, 한 명의 캐릭터를 생성합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**추가 생성된 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        additional_characters = response.text
        additional_characters = re.sub(r'```json\n(.*?)\n```', r'\1', additional_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(additional_characters)
        except Exception:
            new_char = additional_characters
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물 업데이트:\n", self.characters)
        return self.characters

    def create_episode(self) -> str:
        """
        DB에 저장된 이전 에피소드들을 모두 불러와서, 
        이를 기반으로 새 에피소드(초안 또는 다음 화)를 생성합니다.
        각 에피소드는 500-1000자 정도로 작성합니다.
        """
        previous_episodes = self.get_previous_episodes()
        
        if not previous_episodes:
            # 이전 에피소드가 없다면 첫 번째 에피소드(초안) 생성
            instruction = """
            당신은 창의적이고 독창적인 소설 작가입니다. 
            주어진 장르, 제목, 세계관, 줄거리, 등장인물을 기반으로 소설의 초안을 작성해야 합니다.
            각 에피소드는 500-1000자 정도로 작성해야 합니다.

            <작성 지침>
            - 소설의 분위기는 주어진 장르에 맞게 설정하세요.
            - 첫 번째 에피소드에서는 주인공을 등장시키고, 소설의 시작점을 설정하세요.
            - 분량을 준수하되, 에피소드가 끝날 때 문장이 완결되도록 작성하세요.
            """
            episode_label = "**소설 초안**\n"
        else:
            # 이전 에피소드가 있다면 DB의 모든 내용을 기반으로 다음 화 생성
            instruction = """
            당신은 창의적이고 독창적인 소설 작가입니다. 
            주어진 장르, 제목, 세계관, 줄거리, 등장인물, 소설 이전화를 기반으로 소설의 다음화를 작성해야 합니다.
            각 에피소드는 500-1000자 정도로 작성해야 합니다.

            <작성 지침>
            - 소설의 분위기는 주어진 장르에 맞게 설정하세요.
            - 이전화의 내용을 참고하여 내용이 자연스럽게 연결되도록 작성하세요.
            - 분량을 준수하되, 에피소드가 끝날 때 문장이 완결되도록 작성하세요.
            """
            episode_label = "**소설 다음화**\n"
        
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 소설 등장인물: {self.characters}\n"
        )
        if previous_episodes:
            prompt += f"## 소설 이전화: {previous_episodes}\n\n"
        prompt += episode_label

        response = model.generate_content(prompt)
        episode_content = response.text

        # 에피소드 번호는 기존 에피소드 수 + 1
        cursor = self.db_connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM episode WHERE novel_pk = %s", (self.novel_pk,))
        (count,) = cursor.fetchone()
        cursor.close()
        ep_number = count + 1
        ep_title = f"Episode {ep_number}"

        # 새로 생성된 에피소드를 DB에 저장
        self.insert_episode(ep_title, episode_content)

        if ep_number == 1:
            self.first_episode = episode_content
            print("에피소드 1화:\n", episode_content)
        else:
            self.next_episode = episode_content
            print(f"에피소드 {ep_number}화:\n", episode_content)
        return episode_content


## 장르, 제목 입력

In [44]:
# 소설의 장르와 제목을 입력합니다.
genre = input("소설의 장르를 입력하세요: ex) 판타지, 로맨스, 스릴러 등\n")
title = input("소설의 제목을 입력하세요: ex) 괴식식당, 신의탑, 전생전기 등\n")
print(genre)
print(title)

novel_gen = NovelGenerator(genre, title)

판타지, 액션, 일상물
괴식식당


## 세계관 생성

In [45]:
novel_gen.recommend_worldview()

소설 세계관:
 1. 기본 설정

시대: 현대와 유사한 시대이나, 괴수와 마법이 존재하는 평행 세계. 현대 문명과 유사한 기술 수준을 가지고 있지만, 괴수와 마법의 존재로 인해 사회 곳곳에 특수한 기술과 문화가 발달했다. 괴수가 나타나는 빈도와 강도는 지역에 따라 다르며, 이에 따라 사람들의 생활 방식도 달라진다.

핵심 테마: '일상 속 비일상'을 다룬다. 괴수와 마법이 존재하는 세계이지만, 사람들은 나름대로 평범한 일상을 살아간다. 괴수를 사냥하는 헌터, 마법을 연구하는 학자, 괴수의 부산물로 요리를 만드는 요리사 등 다양한 직업들이 존재하며, 이들의 삶 속에서 벌어지는 소소한 이야기들을 통해 세계관을 보여준다. 특히 '괴식식당'은 이러한 비일상적인 요소들을 일상적인 음식이라는 매개체를 통해 풀어내는 중심 공간이다.

2. 지리적 배경

세계 전체의 구조: 지구와 유사한 형태의 행성이지만, 대륙의 분포와 크기가 다르다. 아시아 대륙과 유사한 '동방 대륙', 유럽 대륙과 유사한 '서방 대륙', 아프리카 대륙과 유사한 '남방 대륙', 아메리카 대륙과 유사한 '북방 대륙'이 존재한다. 각 대륙은 고유한 문화와 환경을 가지고 있으며, 괴수의 종류와 분포도 다르다. '괴식식당'은 동방 대륙의 대도시에 위치해 있다.

주요 지형 및 자연 환경:
- 거대한 산맥: 대륙 곳곳에 솟아 있으며, 괴수의 서식지이자 헌터들의 주요 활동 무대이다. 산맥의 정상에는 마력이 강하게 흐르는 지역이 존재하며, 희귀한 약초와 광물이 채취된다.
- 넓은 평원: 농경지와 목초지로 이용되며, 비교적 안전한 지역이다. 하지만 가끔씩 평원을 가로지르는 강력한 괴수가 나타나기도 한다.
- 깊은 숲: 다양한 생물들이 서식하며, 마법사들의 수련 장소로 이용된다. 숲 속에는 고대 유적이나 숨겨진 마을이 존재하기도 한다.
- 해안 도시: 해상 무역의 중심지이며, 해양 괴수의 공격에 대비해야 한다. 해안 도시에서는 독특한 해산물 요리가 발달했다.

상징적이거나 중요한 장소:
- 용의 협곡: 과거 강력한 용이

"1. 기본 설정\n\n시대: 현대와 유사한 시대이나, 괴수와 마법이 존재하는 평행 세계. 현대 문명과 유사한 기술 수준을 가지고 있지만, 괴수와 마법의 존재로 인해 사회 곳곳에 특수한 기술과 문화가 발달했다. 괴수가 나타나는 빈도와 강도는 지역에 따라 다르며, 이에 따라 사람들의 생활 방식도 달라진다.\n\n핵심 테마: '일상 속 비일상'을 다룬다. 괴수와 마법이 존재하는 세계이지만, 사람들은 나름대로 평범한 일상을 살아간다. 괴수를 사냥하는 헌터, 마법을 연구하는 학자, 괴수의 부산물로 요리를 만드는 요리사 등 다양한 직업들이 존재하며, 이들의 삶 속에서 벌어지는 소소한 이야기들을 통해 세계관을 보여준다. 특히 '괴식식당'은 이러한 비일상적인 요소들을 일상적인 음식이라는 매개체를 통해 풀어내는 중심 공간이다.\n\n2. 지리적 배경\n\n세계 전체의 구조: 지구와 유사한 형태의 행성이지만, 대륙의 분포와 크기가 다르다. 아시아 대륙과 유사한 '동방 대륙', 유럽 대륙과 유사한 '서방 대륙', 아프리카 대륙과 유사한 '남방 대륙', 아메리카 대륙과 유사한 '북방 대륙'이 존재한다. 각 대륙은 고유한 문화와 환경을 가지고 있으며, 괴수의 종류와 분포도 다르다. '괴식식당'은 동방 대륙의 대도시에 위치해 있다.\n\n주요 지형 및 자연 환경:\n- 거대한 산맥: 대륙 곳곳에 솟아 있으며, 괴수의 서식지이자 헌터들의 주요 활동 무대이다. 산맥의 정상에는 마력이 강하게 흐르는 지역이 존재하며, 희귀한 약초와 광물이 채취된다.\n- 넓은 평원: 농경지와 목초지로 이용되며, 비교적 안전한 지역이다. 하지만 가끔씩 평원을 가로지르는 강력한 괴수가 나타나기도 한다.\n- 깊은 숲: 다양한 생물들이 서식하며, 마법사들의 수련 장소로 이용된다. 숲 속에는 고대 유적이나 숨겨진 마을이 존재하기도 한다.\n- 해안 도시: 해상 무역의 중심지이며, 해양 괴수의 공격에 대비해야 한다. 해안 도시에서는 독특한 해산물 요리가 발달했다.\n\n상징적이거나 중요한 장소:\n- 용의 협곡: 

## 줄거리 생성

In [46]:
novel_gen.recommend_synopsis()

소설 줄거리:
 1. 기본 정보
    - 장르: 판타지, 액션, 일상물
    - 제목: 괴식식당
    - 세계관: 괴수와 마법이 존재하는 현대와 유사한 평행 세계. 괴수의 부산물로 요리를 만드는 '괴식식당'을 중심으로 이야기가 펼쳐진다.

2. 발단
    - 동방 대륙의 번화한 도시, 겉보기엔 평범한 식당 '괴식식당'이 문을 연다. 주인은 어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년, 강혁이다.
    - 강혁은 손님들에게 단순히 맛있는 음식을 제공하는 것을 넘어, 괴수의 특성을 살린 특별한 요리를 통해 잊지 못할 경험을 선사하고자 한다. 그의 요리는 헌터들 사이에서 입소문을 타기 시작한다.
    - 한편, 도시에는 정체불명의 괴수가 출몰하기 시작하고, 헌터들은 괴수의 흔적을 쫓지만 번번이 실패한다. 강혁은 괴수의 습격이 단순한 우연이 아님을 직감한다.

3. 전개
    - 괴식식당은 헌터들의 아지트가 되어, 강혁은 그들에게 괴수 정보를 제공하고, 헌터들은 강혁에게 괴수 사냥에 대한 이야기를 들려준다. 
    - 강혁은 우연히 마법사, 엘프, 드워프 등 다양한 종족의 손님들과 만나게 되고, 그들과 교류하며 괴식에 대한 새로운 영감을 얻는다.
    - 강혁은 괴수 연구에 몰두하며, 괴수의 부산물에 숨겨진 특별한 효능을 발견한다. 그는 이를 활용하여 더욱 기상천외한 요리를 개발하고, 손님들의 건강과 능력을 향상시키는 데 기여한다.
    - 도시를 습격하는 괴수의 배후에는 어둠의 세력이 있다는 사실이 드러난다. 그들은 괴수를 이용하여 도시를 혼란에 빠뜨리고, 자신들의 목적을 달성하려 한다.

4. 위기
    - 어둠의 세력은 괴식식당을 눈엣가시로 여기고, 강혁을 제거하려 한다. 그들은 식당에 괴수를 풀어 습격하고, 강혁과 헌터들은 힘겹게 괴수를 물리치지만, 식당은 큰 피해를 입는다.
    - 강혁은 어둠의 세력이 과거 괴수 전쟁 당시 패배한 마법사들의 잔당이라는 사실을 알게 된다. 그들은 인간에게 복수하기 위해, 봉인된 

"1. 기본 정보\n    - 장르: 판타지, 액션, 일상물\n    - 제목: 괴식식당\n    - 세계관: 괴수와 마법이 존재하는 현대와 유사한 평행 세계. 괴수의 부산물로 요리를 만드는 '괴식식당'을 중심으로 이야기가 펼쳐진다.\n\n2. 발단\n    - 동방 대륙의 번화한 도시, 겉보기엔 평범한 식당 '괴식식당'이 문을 연다. 주인은 어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년, 강혁이다.\n    - 강혁은 손님들에게 단순히 맛있는 음식을 제공하는 것을 넘어, 괴수의 특성을 살린 특별한 요리를 통해 잊지 못할 경험을 선사하고자 한다. 그의 요리는 헌터들 사이에서 입소문을 타기 시작한다.\n    - 한편, 도시에는 정체불명의 괴수가 출몰하기 시작하고, 헌터들은 괴수의 흔적을 쫓지만 번번이 실패한다. 강혁은 괴수의 습격이 단순한 우연이 아님을 직감한다.\n\n3. 전개\n    - 괴식식당은 헌터들의 아지트가 되어, 강혁은 그들에게 괴수 정보를 제공하고, 헌터들은 강혁에게 괴수 사냥에 대한 이야기를 들려준다. \n    - 강혁은 우연히 마법사, 엘프, 드워프 등 다양한 종족의 손님들과 만나게 되고, 그들과 교류하며 괴식에 대한 새로운 영감을 얻는다.\n    - 강혁은 괴수 연구에 몰두하며, 괴수의 부산물에 숨겨진 특별한 효능을 발견한다. 그는 이를 활용하여 더욱 기상천외한 요리를 개발하고, 손님들의 건강과 능력을 향상시키는 데 기여한다.\n    - 도시를 습격하는 괴수의 배후에는 어둠의 세력이 있다는 사실이 드러난다. 그들은 괴수를 이용하여 도시를 혼란에 빠뜨리고, 자신들의 목적을 달성하려 한다.\n\n4. 위기\n    - 어둠의 세력은 괴식식당을 눈엣가시로 여기고, 강혁을 제거하려 한다. 그들은 식당에 괴수를 풀어 습격하고, 강혁과 헌터들은 힘겹게 괴수를 물리치지만, 식당은 큰 피해를 입는다.\n    - 강혁은 어둠의 세력이 과거 괴수 전쟁 당시 패배한 마법사들의 잔당이라는 사실을 알게 된다. 그들은 인간에게 복수하

## 등장인물 생성

In [47]:
novel_gen.recommend_characters()

소설 등장인물:
 [
  {
    "이름": "강혁",
    "성별": "남",
    "나이": "20대 후반",
    "역할": "주인공",
    "직업": "괴식식당 요리사",
    "프로필": "어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년으로, 180cm의 훤칠한 키에 덥수룩한 머리를 질끈 묶고 다니며, 괴수의 특성을 살린 특별한 요리를 통해 손님들에게 잊지 못할 경험을 선사하고자 한다. 과거 괴수에게 가족을 잃은 슬픔을 딛고 일어섰으며, 요리를 통해 사람들에게 희망을 주고자 한다."
  },
  {
    "이름": "유리아",
    "성별": "여",
    "나이": "20대 초반",
    "역할": "조력자",
    "직업": "헌터",
    "프로필": "뛰어난 사격 솜씨와 냉철한 판단력을 가진 베테랑 헌터로, 170cm의 늘씬한 몸매에 검은색 가죽 갑옷을 입고 다니며, 괴수 사냥에 있어서는 누구보다 냉정하지만, 동료들을 아끼는 따뜻한 마음씨를 가지고 있다. 과거 강혁에게 도움을 받은 적이 있으며, 이후 그의 든든한 조력자가 된다."
  },
  {
    "이름": "아이작",
    "성별": "남",
    "나이": "30대 중반",
    "역할": "조력자",
    "직업": "마법사",
    "프로필": "지적이고 냉철한 성격의 마법사로, 175cm의 마른 체형에 항상 책을 들고 다니며, 마법 연구에 몰두하며, 세상을 이치적으로 이해하려 한다. 강혁의 요리에 숨겨진 마법적인 가능성을 발견하고, 그에게 협력한다."
  },
  {
    "이름": "고블린 장로",
    "성별": "남",
    "나이": "불명 (외형은 70대)",
    "역할": "조력자",
    "직업": "자칭 미식가",
    "프로필": "작은 체구에 꼬부랑 허리를 가진 고블린이지만, 음식에 대한 열정만큼은 누구에게도 뒤지지 않으며, 독특한 미각으로 강혁의 요리에 대한 날카로운 평가를 내린다. 과거 인간에

'[\n  {\n    "이름": "강혁",\n    "성별": "남",\n    "나이": "20대 후반",\n    "역할": "주인공",\n    "직업": "괴식식당 요리사",\n    "프로필": "어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년으로, 180cm의 훤칠한 키에 덥수룩한 머리를 질끈 묶고 다니며, 괴수의 특성을 살린 특별한 요리를 통해 손님들에게 잊지 못할 경험을 선사하고자 한다. 과거 괴수에게 가족을 잃은 슬픔을 딛고 일어섰으며, 요리를 통해 사람들에게 희망을 주고자 한다."\n  },\n  {\n    "이름": "유리아",\n    "성별": "여",\n    "나이": "20대 초반",\n    "역할": "조력자",\n    "직업": "헌터",\n    "프로필": "뛰어난 사격 솜씨와 냉철한 판단력을 가진 베테랑 헌터로, 170cm의 늘씬한 몸매에 검은색 가죽 갑옷을 입고 다니며, 괴수 사냥에 있어서는 누구보다 냉정하지만, 동료들을 아끼는 따뜻한 마음씨를 가지고 있다. 과거 강혁에게 도움을 받은 적이 있으며, 이후 그의 든든한 조력자가 된다."\n  },\n  {\n    "이름": "아이작",\n    "성별": "남",\n    "나이": "30대 중반",\n    "역할": "조력자",\n    "직업": "마법사",\n    "프로필": "지적이고 냉철한 성격의 마법사로, 175cm의 마른 체형에 항상 책을 들고 다니며, 마법 연구에 몰두하며, 세상을 이치적으로 이해하려 한다. 강혁의 요리에 숨겨진 마법적인 가능성을 발견하고, 그에게 협력한다."\n  },\n  {\n    "이름": "고블린 장로",\n    "성별": "남",\n    "나이": "불명 (외형은 70대)",\n    "역할": "조력자",\n    "직업": "자칭 미식가",\n    "프로필": "작은 체구에 꼬부랑 허리를 가진 고블린이지만, 음식에 대한 열정만큼은 누구에게도 뒤지지 않으며, 독특한 미각으로 강혁의 요리에 대

## 추가 등장인물 생성

In [48]:
# 추가 등장인물 생성을 위한 기능 호출
novel_gen.add_new_characters()

소설 등장인물 업데이트:
 [
  {
    "이름": "강혁",
    "성별": "남",
    "나이": "20대 후반",
    "역할": "주인공",
    "직업": "괴식식당 요리사",
    "프로필": "어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년으로, 180cm의 훤칠한 키에 덥수룩한 머리를 질끈 묶고 다니며, 괴수의 특성을 살린 특별한 요리를 통해 손님들에게 잊지 못할 경험을 선사하고자 한다. 과거 괴수에게 가족을 잃은 슬픔을 딛고 일어섰으며, 요리를 통해 사람들에게 희망을 주고자 한다."
  },
  {
    "이름": "유리아",
    "성별": "여",
    "나이": "20대 초반",
    "역할": "조력자",
    "직업": "헌터",
    "프로필": "뛰어난 사격 솜씨와 냉철한 판단력을 가진 베테랑 헌터로, 170cm의 늘씬한 몸매에 검은색 가죽 갑옷을 입고 다니며, 괴수 사냥에 있어서는 누구보다 냉정하지만, 동료들을 아끼는 따뜻한 마음씨를 가지고 있다. 과거 강혁에게 도움을 받은 적이 있으며, 이후 그의 든든한 조력자가 된다."
  },
  {
    "이름": "아이작",
    "성별": "남",
    "나이": "30대 중반",
    "역할": "조력자",
    "직업": "마법사",
    "프로필": "지적이고 냉철한 성격의 마법사로, 175cm의 마른 체형에 항상 책을 들고 다니며, 마법 연구에 몰두하며, 세상을 이치적으로 이해하려 한다. 강혁의 요리에 숨겨진 마법적인 가능성을 발견하고, 그에게 협력한다."
  },
  {
    "이름": "고블린 장로",
    "성별": "남",
    "나이": "불명 (외형은 70대)",
    "역할": "조력자",
    "직업": "자칭 미식가",
    "프로필": "작은 체구에 꼬부랑 허리를 가진 고블린이지만, 음식에 대한 열정만큼은 누구에게도 뒤지지 않으며, 독특한 미각으로 강혁의 요리에 대한 날카로운 평가를 내린다. 과

'[\n  {\n    "이름": "강혁",\n    "성별": "남",\n    "나이": "20대 후반",\n    "역할": "주인공",\n    "직업": "괴식식당 요리사",\n    "프로필": "어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년으로, 180cm의 훤칠한 키에 덥수룩한 머리를 질끈 묶고 다니며, 괴수의 특성을 살린 특별한 요리를 통해 손님들에게 잊지 못할 경험을 선사하고자 한다. 과거 괴수에게 가족을 잃은 슬픔을 딛고 일어섰으며, 요리를 통해 사람들에게 희망을 주고자 한다."\n  },\n  {\n    "이름": "유리아",\n    "성별": "여",\n    "나이": "20대 초반",\n    "역할": "조력자",\n    "직업": "헌터",\n    "프로필": "뛰어난 사격 솜씨와 냉철한 판단력을 가진 베테랑 헌터로, 170cm의 늘씬한 몸매에 검은색 가죽 갑옷을 입고 다니며, 괴수 사냥에 있어서는 누구보다 냉정하지만, 동료들을 아끼는 따뜻한 마음씨를 가지고 있다. 과거 강혁에게 도움을 받은 적이 있으며, 이후 그의 든든한 조력자가 된다."\n  },\n  {\n    "이름": "아이작",\n    "성별": "남",\n    "나이": "30대 중반",\n    "역할": "조력자",\n    "직업": "마법사",\n    "프로필": "지적이고 냉철한 성격의 마법사로, 175cm의 마른 체형에 항상 책을 들고 다니며, 마법 연구에 몰두하며, 세상을 이치적으로 이해하려 한다. 강혁의 요리에 숨겨진 마법적인 가능성을 발견하고, 그에게 협력한다."\n  },\n  {\n    "이름": "고블린 장로",\n    "성별": "남",\n    "나이": "불명 (외형은 70대)",\n    "역할": "조력자",\n    "직업": "자칭 미식가",\n    "프로필": "작은 체구에 꼬부랑 허리를 가진 고블린이지만, 음식에 대한 열정만큼은 누구에게도 뒤지지 않으며, 독특한 미각으로 강혁의 요리에 대

In [49]:
# 추가 등장인물 생성을 위한 기능 호출
novel_gen.add_new_characters()

소설 등장인물 업데이트:
 [
  {
    "이름": "강혁",
    "성별": "남",
    "나이": "20대 후반",
    "역할": "주인공",
    "직업": "괴식식당 요리사",
    "프로필": "어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년으로, 180cm의 훤칠한 키에 덥수룩한 머리를 질끈 묶고 다니며, 괴수의 특성을 살린 특별한 요리를 통해 손님들에게 잊지 못할 경험을 선사하고자 한다. 과거 괴수에게 가족을 잃은 슬픔을 딛고 일어섰으며, 요리를 통해 사람들에게 희망을 주고자 한다."
  },
  {
    "이름": "유리아",
    "성별": "여",
    "나이": "20대 초반",
    "역할": "조력자",
    "직업": "헌터",
    "프로필": "뛰어난 사격 솜씨와 냉철한 판단력을 가진 베테랑 헌터로, 170cm의 늘씬한 몸매에 검은색 가죽 갑옷을 입고 다니며, 괴수 사냥에 있어서는 누구보다 냉정하지만, 동료들을 아끼는 따뜻한 마음씨를 가지고 있다. 과거 강혁에게 도움을 받은 적이 있으며, 이후 그의 든든한 조력자가 된다."
  },
  {
    "이름": "아이작",
    "성별": "남",
    "나이": "30대 중반",
    "역할": "조력자",
    "직업": "마법사",
    "프로필": "지적이고 냉철한 성격의 마법사로, 175cm의 마른 체형에 항상 책을 들고 다니며, 마법 연구에 몰두하며, 세상을 이치적으로 이해하려 한다. 강혁의 요리에 숨겨진 마법적인 가능성을 발견하고, 그에게 협력한다."
  },
  {
    "이름": "고블린 장로",
    "성별": "남",
    "나이": "불명 (외형은 70대)",
    "역할": "조력자",
    "직업": "자칭 미식가",
    "프로필": "작은 체구에 꼬부랑 허리를 가진 고블린이지만, 음식에 대한 열정만큼은 누구에게도 뒤지지 않으며, 독특한 미각으로 강혁의 요리에 대한 날카로운 평가를 내린다. 과

'[\n  {\n    "이름": "강혁",\n    "성별": "남",\n    "나이": "20대 후반",\n    "역할": "주인공",\n    "직업": "괴식식당 요리사",\n    "프로필": "어릴 적부터 괴수에 대한 해박한 지식과 뛰어난 요리 솜씨를 가진 청년으로, 180cm의 훤칠한 키에 덥수룩한 머리를 질끈 묶고 다니며, 괴수의 특성을 살린 특별한 요리를 통해 손님들에게 잊지 못할 경험을 선사하고자 한다. 과거 괴수에게 가족을 잃은 슬픔을 딛고 일어섰으며, 요리를 통해 사람들에게 희망을 주고자 한다."\n  },\n  {\n    "이름": "유리아",\n    "성별": "여",\n    "나이": "20대 초반",\n    "역할": "조력자",\n    "직업": "헌터",\n    "프로필": "뛰어난 사격 솜씨와 냉철한 판단력을 가진 베테랑 헌터로, 170cm의 늘씬한 몸매에 검은색 가죽 갑옷을 입고 다니며, 괴수 사냥에 있어서는 누구보다 냉정하지만, 동료들을 아끼는 따뜻한 마음씨를 가지고 있다. 과거 강혁에게 도움을 받은 적이 있으며, 이후 그의 든든한 조력자가 된다."\n  },\n  {\n    "이름": "아이작",\n    "성별": "남",\n    "나이": "30대 중반",\n    "역할": "조력자",\n    "직업": "마법사",\n    "프로필": "지적이고 냉철한 성격의 마법사로, 175cm의 마른 체형에 항상 책을 들고 다니며, 마법 연구에 몰두하며, 세상을 이치적으로 이해하려 한다. 강혁의 요리에 숨겨진 마법적인 가능성을 발견하고, 그에게 협력한다."\n  },\n  {\n    "이름": "고블린 장로",\n    "성별": "남",\n    "나이": "불명 (외형은 70대)",\n    "역할": "조력자",\n    "직업": "자칭 미식가",\n    "프로필": "작은 체구에 꼬부랑 허리를 가진 고블린이지만, 음식에 대한 열정만큼은 누구에게도 뒤지지 않으며, 독특한 미각으로 강혁의 요리에 대

## 초안 생성

In [50]:
# 첫 번째 에피소드(초안) 생성
episode1 = novel_gen.create_episode()

에피소드 1화:
 ## 괴식식당 1화

동방 대륙의 심장부, 네온사인이 현란하게 빛나는 거대한 도시 ‘신룡’의 뒷골목. 낡은 간판에 삐뚤빼뚤하게 적힌 ‘괴식식당’이란 네 글자가 어색하게 빛나고 있었다. 문을 열자, 코를 찌르는 듯하면서도 묘하게 식욕을 자극하는 기이한 향이 후각을 자극했다.

주방에서는 덥수룩한 머리를 질끈 묶은 청년, 강혁이 능숙한 손놀림으로 요리에 열중하고 있었다. 그의 주변에는 낯선 재료들이 가득했다. 반투명한 껍질을 가진 괴수 달걀, 꿈틀거리는 지네처럼 생긴 풀, 형체를 알아볼 수 없는 기괴한 고기 덩어리까지. 여느 식당에서는 구경조차 할 수 없는 재료들이었다.

"흐음, 오늘은 뭘 만들어볼까."

강혁은 싱싱한 괴수 달걀을 깨뜨려 놋쇠 그릇에 담았다. 뽀얀 흰자와 검붉은 노른자가 섞이는 모습은 어딘가 기괴하면서도 아름다웠다. 그는 칼을 들어 옆에 놓인 괴수 고기 덩어리를 큼지막하게 썰어 넣었다. 칼날이 고기를 가를 때마다 뿜어져 나오는 푸르스름한 액체가 마치 독극물처럼 보였다.

“손님, 주문하시겠어요?”

손님은 검은 가죽 갑옷을 입은 날카로운 인상의 여헌터, 유리아였다. 그녀는 묵묵히 메뉴판을 훑어보더니, 무심하게 입을 열었다.

"오늘의 추천 요리."

"오늘의 추천은... '지옥불 도마뱀 꼬치'입니다. 매운맛을 좋아하는 분께는 최고의 선택이죠."

강혁은 싱긋 웃으며 꼬치를 굽기 시작했다. 도마뱀 꼬치는 붉은색의 특수한 소스에 발라져 있었는데, 불판에 올려지자마자 매캐한 연기를 뿜어내며 지옥불처럼 타올랐다. 유리아는 흥미로운 눈빛으로 그 모습을 지켜봤다.

잠시 후, 강혁은 김이 모락모락 나는 도마뱀 꼬치를 유리아에게 건넸다. 꼬치에서는 강렬한 매운 향이 코를 찔렀다. 유리아는 망설임 없이 꼬치를 입으로 가져갔다.

“...!”

매운맛이 폭발하듯 입안을 강타했다. 유리아는 저절로 눈을 가늘게 떴다. 하지만 단순히 매운맛만이 아니었다. 꼬치 안에는 쫄깃한 도마뱀 살코기가 숨어 있었고, 씹을수록 은은한 단맛과 고소한 기름이 흘러나

## 다음화 생성

In [51]:
# 이후 생성할 에피소드는 create_episode()를 호출하면 DB에 저장된 모든 에피소드가 입력값으로 사용됩니다.
episode2 = novel_gen.create_episode()

에피소드 2화:
 ## 괴식식당 2화

유리아는 강혁의 말에 즉각적으로 반응했다. 그녀의 손은 이미 허리춤에 찬 검을 향해 움직이고 있었다. "어디에 나타났나?" 그녀의 목소리는 평소의 무심함과는 달리, 날카로운 긴장감이 감돌았다.

"동쪽, 낡은 창고 지구 쪽에... 놈이 꽤 큰 것 같습니다! 벌써 몇 명이 다쳤다고..." 남자는 숨을 고르며 다급하게 상황을 전했다. 그의 얼굴은 공포에 질려 창백하게 질려 있었다.

강혁은 앞치마를 벗어 던지며 유리아에게 짧게 말했다. "가시죠. 뒷일은 제가 알아서 하겠습니다." 그는 익숙하게 칼을 챙겨 허리춤에 찼다. 그의 눈빛은 차분했지만, 그 속에는 단단한 결의가 담겨 있었다.

식당을 나선 강혁과 유리아는 도시의 번잡한 거리를 빠르게 가로질렀다. 네온사인이 화려하게 빛나는 도시의 밤거리와는 대조적으로, 창고 지구는 어둡고 음산한 분위기를 풍겼다. 낡은 건물들은 마치 거대한 괴물의 이빨처럼 삐죽삐죽 솟아 있었고, 골목길은 마치 미로처럼 얽혀 있었다.

창고 지구에 가까워질수록, 불안한 웅성거림과 비명 소리가 더욱 크게 들려왔다. 곧, 그들은 거대한 괴수의 모습을 목격할 수 있었다. 그것은 마치 거대한 바퀴벌레와 늑대를 섞어놓은 듯한 흉측한 모습이었다. 놈의 몸은 단단한 갑피로 덮여 있었고, 날카로운 발톱과 이빨은 보는 이들을 압도하는 위압감을 풍겼다.

"젠장, '강철 이빨 늑대벌레'잖아!" 유리아는 낮게 읊조렸다. "저 녀석은 갑피가 워낙 단단해서, 일반적인 공격으로는 상처조차 내기 힘들어..."

강철 이빨 늑대벌레는 닥치는 대로 건물을 부수고 사람들을 공격하고 있었다. 헌터들이 놈에게 달려들었지만, 역부족이었다. 놈의 강력한 발톱 공격에 나가떨어지거나, 날카로운 이빨에 물려 쓰러지는 헌터들이 속출했다.

강혁은 잠시 상황을 살피더니, 유리아에게 말했다. "정면 공격은 무모합니다. 놈의 약점을 찾아야 합니다."

"약점? 저 녀석에게 약점 따위가 있을 리가 없잖아!" 유리아는 절망적인 표정으로 외쳤다.

"있습니다.

In [52]:
episode3 = novel_gen.create_episode()

에피소드 3화:
 ## 괴식식당 3화

유리아는 강혁의 심각한 표정을 보며 걱정스러운 눈빛을 보냈다. "너무 깊게 생각하지 마. 어쨌든 오늘은 우리가 이겼잖아." 그녀는 억지로 밝은 목소리를 내며 강혁을 위로하려 했다.

하지만 강혁의 불안감은 쉽게 가시지 않았다. 그는 늑대벌레의 시체를 더욱 자세히 살펴보았다. 놈의 몸 곳곳에는 인위적인 상처들이 남아 있었다. 마치 누군가가 실험을 한 것처럼, 억지로 괴수의 힘을 끌어올리려 한 흔적이었다.

"이 녀석, 누군가에게 조종당하고 있었어." 강혁은 확신에 찬 목소리로 말했다. "그리고, 그 힘이 불안정해서 약해진 거야."

유리아는 미간을 찌푸리며 질문했다. "조종? 설마... 어둠의 마법사들이 또다시 나타난 건가?"

"가능성은 충분해." 강혁은 고개를 끄덕였다. "과거 괴수 전쟁에서 패배한 어둠의 마법사들은 인간에 대한 복수심을 품고, 봉인된 괴수들을 깨우려 할 거야."

두 사람은 잠시 침묵에 잠겼다. 도시의 평화는 한순간에 깨질 수 있었다. 어둠의 세력이 다시 활동을 시작한다면, 신룡은 걷잡을 수 없는 혼란에 빠질 것이다.

"일단 헌터 길드에 이 사실을 알려야 해." 유리아는 결심한 듯 말했다. "그리고, 우리도 나름대로 조사를 시작해야 해."

"좋아." 강혁은 동의하며 말했다. "저는 식당에서 정보를 수집하고, 당신은 헌터 길드와 협력해 주세요."

두 사람은 각자의 역할을 분담하고, 다시 한번 괴식식당으로 향했다. 식당은 아까의 소란으로 인해 엉망진창이 되어 있었다. 깨진 유리 조각과 엎어진 테이블들이 당시의 긴박했던 상황을 그대로 보여주고 있었다.

강혁은 한숨을 쉬며 식당 정리를 시작했다. 그는 손님들에게 맛있는 요리를 제공하는 것만큼이나, 식당을 깨끗하게 유지하는 것을 중요하게 생각했다. 빗자루로 바닥을 쓸고, 걸레로 테이블을 닦는 동안에도 그의 머릿속에는 어둠의 세력에 대한 생각으로 가득 차 있었다.

그때, 식당 문이 조용히 열리며 한 남자가 들어왔다. 그는 낡은 로브를 입고, 얼굴을 가리는

In [53]:
episode4 = novel_gen.create_episode()

에피소드 4화:
 ## 괴식식당 4화

유리아의 다급한 목소리에 식당 안의 긴장감이 다시 고조되었다. 강혁은 아이작에게 잠시 양해를 구하고 유리아에게 다가갔다. "무슨 일이야?" 그의 목소리는 낮았지만, 심각한 분위기를 감추지 못했다.

"헌터 길드에 따르면, 도시 곳곳에서 괴수 출몰이 잇따르고 있대. 그것도 평소에는 보기 힘든 강력한 개체들이." 유리아는 숨을 고르며 말했다. "게다가, 괴수들이 특정한 장소를 향해 움직이고 있다는 정보도 입수했어."

"특정한 장소라..." 강혁은 턱을 매만지며 생각에 잠겼다. "혹시 그 장소가 어디인지 알아?"

"아직 정확한 위치는 파악하지 못했지만, 공통점은 있어. 모두 과거 괴수 전쟁의 격전지였다는 거야." 유리아는 심각한 표정으로 대답했다. "어둠의 세력이 그곳에 봉인된 괴수들을 깨우려는 게 분명해."

강혁은 유리아의 말을 듣고 무언가를 깨달은 듯 눈을 크게 떴다. "설마... 놈들이 노리는 건 '용의 협곡'인가?"

"용의 협곡?" 유리아는 고개를 갸웃거렸다. "그곳은 과거 강력한 용이 살았던 곳이라고는 하지만, 지금은 폐허나 다름없잖아. 어째서 그런 곳을..."

"놈들은 용의 기운을 이용하려는 거야." 강혁은 확신에 찬 목소리로 말했다. "용의 협곡에는 아직 용의 힘이 남아있어. 어둠의 마법사들은 그 힘을 증폭시켜 봉인을 풀고, 더욱 강력한 괴수를 소환하려는 거지."

아이작은 두 사람의 대화를 주의 깊게 듣고 있었다. 그는 지팡이를 짚으며 강혁에게 다가왔다. "용의 협곡이라... 흥미로운 곳이군요. 저도 함께 가도 될까요?"

"마법사님께서요?" 강혁은 의외라는 듯 아이작을 바라봤다. "위험할 수도 있습니다."

"저는 호기심이 많은 마법사일 뿐입니다." 아이작은 미소를 지으며 대답했다. "게다가, 당신의 요리에 담긴 힘을 연구하기 위해서는, 괴수가 있는 곳으로 가는 것이 가장 좋은 방법이겠죠."

강혁은 잠시 고민하더니, 아이작의 제안을 수락했다. "좋습니다. 함께 가시죠. 하지만, 위험한 상황이 발생하면

In [54]:
episode5 = novel_gen.create_episode()

에피소드 5화:
 ## 괴식식당 5화

아이작의 외침에 유리아와 강혁은 동시에 의아한 표정을 지었다. “선택받은 자? 그게 무슨 소리죠?” 유리아가 되물었다. 하지만 아이작은 대답 대신 놀라움과 경외가 뒤섞인 시선으로 강혁에게서 눈을 떼지 못했다.

강혁의 몸에서 뿜어져 나오는 빛은 점점 강렬해졌다. 그의 주변에는 형체를 알아볼 수 없는 기운들이 소용돌이치며 마치 보호막처럼 그를 감쌌다. 어둠의 마법사들은 갑작스러운 상황 변화에 당황하며 공격을 멈췄다. 그들의 눈에는 두려움과 경계심이 가득했다.

“저 녀석… 뭔가 심상치 않아!” 한 어둠의 마법사가 떨리는 목소리로 외쳤다. “모두 덤벼! 저 힘이 완전히 깨어나기 전에 없애버려야 해!”

어둠의 마법사들은 다시 한번 주문을 외우며 강혁에게 공격 마법을 퍼부었다. 불덩이, 얼음 조각, 번개 등 다양한 마법들이 쏟아져 내렸지만, 강혁을 감싼 기운들은 마치 무형의 방패처럼 모든 공격을 막아냈다.

강혁은 눈을 감고 자신의 몸 안에서 끓어오르는 힘에 집중했다. 그것은 마치 오래전부터 잠들어 있던 거대한 용이 깨어나는 듯한 느낌이었다. 그는 그 힘이 어디에서 왔는지, 어떻게 사용해야 하는지 알지 못했지만, 본능적으로 그것이 자신에게 주어진 사명이라는 것을 깨달았다.

“그래… 나는 할 수 있어.” 강혁은 나지막이 읊조리며 눈을 떴다. 그의 눈동자는 평소의 갈색빛이 아닌, 짙은 금색으로 빛나고 있었다. 그의 얼굴에는 굳은 결의와 함께 희미한 미소가 떠올랐다.

강혁은 손을 뻗어 허리춤에 찬 칼을 뽑아 들었다. 칼날은 그의 몸에서 뿜어져 나오는 빛을 받아 더욱 날카롭게 빛났다. 그는 칼에 힘을 집중시키자, 칼날에서 푸른색 오라가 뿜어져 나왔다. 그것은 마치 살아있는 괴수의 기운처럼 맹렬하게 타올랐다.

“이제, 끝을 내주지.” 강혁은 낮고 웅장한 목소리로 선언했다. 그의 목소리는 마치 용의 울음소리처럼 협곡 전체에 울려 퍼졌다.

강혁은 칼을 휘두르며 어둠의 마법사들에게 달려들었다. 그의 움직임은 마치 바람처럼 빠르고 날카

In [55]:
episode6 = novel_gen.create_episode()

에피소드 6화:
 ## 괴식식당 6화

신룡으로 돌아오는 길, 강혁은 어쩐지 모를 허탈감에 휩싸였다. 용의 협곡에서 겪었던 일들은 마치 꿈결처럼 느껴졌다. 자신의 몸에서 뿜어져 나왔던 기이한 힘, 어둠의 마법사들과의 격렬한 전투, 그리고 봉인석을 안정화시켰던 순간들. 그는 여전히 그 힘의 정체가 무엇인지 알지 못했지만, 그것이 자신에게 주어진 특별한 능력이라는 것을 어렴풋이 짐작하고 있었다.

"무슨 생각을 그렇게 해?" 유리아가 옆에서 말을 걸었다. 그녀는 강혁의 복잡한 표정을 눈치챈 듯했다. "혹시 아직도 용의 협곡에서의 일이 신경 쓰이는 거야?"

"솔직히 말하면, 잘 모르겠어." 강혁은 솔직하게 털어놓았다. "내가 왜 그런 힘을 가지고 있는지, 앞으로 어떻게 해야 할지... 모든 게 혼란스러워."

"너무 자책하지 마." 유리아는 부드러운 목소리로 위로했다. "네가 어떤 힘을 가지고 있든, 우리는 항상 네 곁에 있을 거야. 함께 고민하고, 함께 헤쳐나가면 돼."

"고마워." 강혁은 유리아의 따뜻한 말에 힘을 얻었다. 그는 그녀의 어깨에 기대어 잠시 눈을 감았다. 그의 마음속에는 여전히 불안감이 남아 있었지만, 동료들이 곁에 있다는 사실이 그에게 큰 위안이 되었다.

아이작은 묵묵히 앞서 걸어가며, 무언가를 골똘히 생각하는 듯했다. 그는 가끔씩 마법 지팡이를 휘두르며 주변의 마력을 측정하거나, 고개를 끄덕이며 혼잣말을 중얼거리기도 했다.

"아이작, 무슨 생각을 그렇게 해?" 강혁이 궁금한 듯 질문했다.

"아, 죄송합니다." 아이작은 잠시 생각에 잠겼다가, 황급히 대답했다. "저는 당신의 힘에 대해 조사하고 있었습니다. 용의 협곡에서 당신이 보여준 힘은 단순한 마력이라고는 할 수 없습니다. 그것은 마치 고대의 유물에 담긴 힘과 유사한, 매우 특별한 에너지입니다."

"고대의 유물?" 강혁은 더욱 혼란스러워졌다. "그게 무슨 뜻이야?"

"정확한 것은 더 조사를 해봐야 알 수 있겠지만, 제 추측으로는 당신은 고대 영웅의 후손일 가능성이 있습니다." 아이

In [56]:
episode7 = novel_gen.create_episode()

에피소드 7화:
 ---

## 괴식식당 7화

강혁은 냉장고에서 신선한 괴수 재료들을 꺼내 능숙하게 손질하기 시작했다. 그의 손놀림은 마치 숙련된 무예가의 검술처럼 빠르고 정확했다. 유리아는 식탁에 앉아 헌터 길드에서 받은 보고서를 검토했고, 아이작은 식당 한 켠에 자리를 잡고 마도구를 꺼내 연구에 몰두했다. 고블린 장로는 흥미로운 듯 강혁의 요리 과정을 지켜보며, 가끔씩 날카로운 질문을 던지기도 했다.

"오늘은 무슨 요리를 만들 셈인가? 혹시 새로운 괴수를 사냥해 온 건가?" 고블린 장로는 호기심 어린 눈빛으로 강혁을 바라봤다.

"오늘은 특별한 손님들을 위한 특별한 요리를 만들 겁니다." 강혁은 미소를 지으며 대답했다. "새로운 괴수를 사냥한 건 아니지만, 아주 희귀한 재료를 구했거든요."

강혁은 냉장고 깊숙한 곳에서 작은 유리병을 꺼냈다. 병 안에는 반짝이는 푸른색 액체가 담겨 있었다. 그것은 바로 용의 협곡에서 얻은 '용의 눈물'이었다. 용의 눈물은 용의 기운이 응축된 매우 희귀한 재료로, 뛰어난 효능을 가지고 있다고 알려져 있었다.

"오오! 용의 눈물이라니! 정말 귀한 것을 구했구먼!" 고블린 장로는 감탄하며 외쳤다. "그것으로 어떤 요리를 만들 셈인가?"

"아직 정하지 못했습니다." 강혁은 곰곰이 생각하며 대답했다. "하지만, 분명 손님들의 몸과 마음에 큰 힘을 줄 수 있는 요리가 될 겁니다."

강혁은 용의 눈물을 조심스럽게 꺼내 놋쇠 그릇에 담았다. 푸른색 액체는 은은한 빛을 내며 식당 안을 신비로운 분위기로 물들였다. 그는 칼을 들어 옆에 놓인 괴수 고기 덩어리를 얇게 썰어 넣었다. 고기는 용의 눈물에 닿자마자 푸른색으로 변하며, 더욱 신선하게 빛났다.

강혁은 놋쇠 그릇에 각종 향신료와 약초를 넣고, 은은한 불에 올려 끓이기 시작했다. 그는 끊임없이 숟가락으로 저으며, 재료들이 서로 조화롭게 어우러지도록 정성을 쏟았다. 그의 집중력은 마치 명상에 잠긴 수도승처럼 고요하고 깊었다.

잠시 후, 식당 안에는 기이하면서도 매혹적인 향이 가